# Random-to-Random

In [1]:
import random as r
import itertools as it
from typing import *

In [2]:
def get_deck(n):
    return list(range(1, n + 1))

In [4]:
class Node:
    def __init__(self, deck:list, marked:Set=set(), depth:int=0, probability:float=1, parent=None):
        self.deck = deck
        self.n = len(deck)
        self.marked = marked
        self.children = []
        self.depth = depth # number of shuffles that have been made
        self.probability = probability # probability of chosing this node from among its siblings
        self.parent = parent
        
    def set_deck(self, deck:List):
        self.deck = deck
    
    def set_marked(self, marked:Set):
        self.marked = marked
        
    def iterate_children(self):
        for child in self.children:
            yield child
        return None
    
    def __str__(self, indent=0):
        return (" " * (indent * 6)) + ("\n" + (" " * (indent * 6))).join([
            "_" * (self.n * 2 + 1),
            "|" + "|".join([" ", "*"][card in self.marked] for card in self.deck) + "|" + f"   depth: {self.depth}",
            "|" + "|".join(str(card) for card in self.deck) + "|" + f"    prob: {self.probability}",
            "‾" * (self.n * 2 + 1),
        ])
    
    def __repr__(self):
        return str(self)
    
    def generate_children(self, shuffle:str):
        """
        make a copy of the node for each possible shuffle
        put those copies in the children of the current node
        compute and record the transition probabilities for each
        """
        self.children = []
        
        if shuffle == "rtr":
            """
            - Remove a card u.a.r. [n choices]
            - Insert it u.a.r. [n choices]
            """
            for position, card in enumerate(self.deck):
                for new_loc in range(n):
                    new_deck = self.deck.copy()
                    new_deck.remove(card)
                    new_deck.insert(new_loc, card)
                    new_marked = self.marked.copy()
                    new_marked.add(card)
                    node = Node(new_deck, new_marked, self.depth + 1, 1 / (self.n * self.n), self)
                    self.children.append(node)
            self.children = self.children[::-1] # to fix annoying ordering
                    
        if shuffle == "modified_rtr_1":
            """
            - Remove a card u.a.r (n choices)
            - Insert it u.a.r in any position except below it (or on the top if the card removes was at the bottom) [n - 1 choices]
            """
            for position, card in enumerate(self.deck):
                for pre_new_loc in range(position + 2, position + 1 + n):
                    new_deck = self.deck.copy()
                    new_deck.remove(card)
                    new_loc = pre_new_loc % n
                    new_deck.insert(new_loc, card)
                    new_marked = self.marked.copy()
                    new_marked.add(card)
                    node = Node(new_deck, new_marked, self.depth + 1, 1 / (self.n * (self.n - 1)), self)
                    self.children.append(node)
            self.children = self.children[::-1] # to fix annoying ordering

        if shuffle == "modified_rtr_2":
            """
            uniform generators
            """
            size_of_S = self.n + (self.n - 2) * (self.n - 1)
            # id
            self.children.append(Node(self.deck.copy(), self.marked.copy(), self.depth + 1, 1 / size_of_S, self))
            # transpositions
            for i in range(self.n - 1):
                new_deck = self.deck.copy()
                new_marked = self.marked.copy()
                if new_deck[i] in new_marked:
                    new_marked.add(new_deck[i + 1])
                elif new_deck[i + 1] in new_marked:
                    new_marked.add(new_deck[i])
#                 else:
#                     new_marked.add(new_deck[i])
                new_deck[i], new_deck[i + 1] = new_deck[i + 1], new_deck[i]
                self.children.append(Node(new_deck, new_marked, self.depth + 1, 1 / size_of_S, self))
            # cycles
            for length in range(3, self.n + 1):
                for initial in range(self.n - length + 1):
                    for direction in [1, -1]:
                        new_deck = self.deck.copy()
                        new_marked = self.marked.copy()
                        a, b = (initial, initial + length - 1)[::direction]
                        card = new_deck[b]
                        new_marked.add(card)
                        new_deck.remove(card)
                        new_deck.insert(a, card)
                        self.children.append(Node(new_deck, new_marked, self.depth + 1, 1 / size_of_S, self))

In [5]:
node = Node([1, 2, 3, 4, 5])
node.generate_children("modified_rtr_2")

In [6]:
print(len(node.children))
node.children

17


[___________
 | | | | | |   depth: 1
 |1|2|3|4|5|    prob: 0.058823529411764705
 ‾‾‾‾‾‾‾‾‾‾‾,
 ___________
 | | | | | |   depth: 1
 |2|1|3|4|5|    prob: 0.058823529411764705
 ‾‾‾‾‾‾‾‾‾‾‾,
 ___________
 | | | | | |   depth: 1
 |1|3|2|4|5|    prob: 0.058823529411764705
 ‾‾‾‾‾‾‾‾‾‾‾,
 ___________
 | | | | | |   depth: 1
 |1|2|4|3|5|    prob: 0.058823529411764705
 ‾‾‾‾‾‾‾‾‾‾‾,
 ___________
 | | | | | |   depth: 1
 |1|2|3|5|4|    prob: 0.058823529411764705
 ‾‾‾‾‾‾‾‾‾‾‾,
 ___________
 |*| | | | |   depth: 1
 |3|1|2|4|5|    prob: 0.058823529411764705
 ‾‾‾‾‾‾‾‾‾‾‾,
 ___________
 | | |*| | |   depth: 1
 |2|3|1|4|5|    prob: 0.058823529411764705
 ‾‾‾‾‾‾‾‾‾‾‾,
 ___________
 | |*| | | |   depth: 1
 |1|4|2|3|5|    prob: 0.058823529411764705
 ‾‾‾‾‾‾‾‾‾‾‾,
 ___________
 | | | |*| |   depth: 1
 |1|3|4|2|5|    prob: 0.058823529411764705
 ‾‾‾‾‾‾‾‾‾‾‾,
 ___________
 | | |*| | |   depth: 1
 |1|2|5|3|4|    prob: 0.058823529411764705
 ‾‾‾‾‾‾‾‾‾‾‾,
 ___________
 | | | | |*|   depth: 1
 |1|2|4|5|3|    prob: 0

In [7]:
class Tree:
    def __init__(self, n:int, shuffle:str):
        self.n = n
        self.top = Node(get_deck(n))
        self.shuffle = shuffle
    
    
    def __str__(self, max_depth:int=None):
        ret = ""
        stack = [self.top]
        while len(stack) > 0:
            node = stack.pop()
            if max_depth is not None and node.depth > max_depth:
                continue
            ret += node.__str__(node.depth) + "\n"
            for child in node.children:
                stack.append(child)
        return ret
    
    
    def __repr__(self):
        return str(self)
    
    
    def print(self, max_depth:int=None):
        print(self.__str__(max_depth))
    
    
    def expand(self, max_depth:int):
        stack = [self.top]
        while len(stack) > 0:
            node = stack.pop()
            if node.depth >= max_depth:
                continue
            if node.children == []:
                node.generate_children(self.shuffle)
            for child in node.children:
                stack.append(child)
        
        
    def get_leaves(self, depth:int, full:bool=False):
        stack = [self.top]
        while len(stack) > 0:
            node = stack.pop()
            if node.depth == depth:
                if not full or len(node.marked) == self.n:
                    yield node
            else:
                for child in node.children:
                    stack.append(child)
        return None
    
    
    def count_full_leaves(self, depth:int):
        """
        counts the occurances of each element of S_n at _depth_ that are full
        """
        self.expand(depth)
        data = {str(list(deck)):0 for deck in it.permutations(range(1, self.n + 1))}
        for node in self.get_leaves(depth, full=True):
            data[str(node.deck)] += 1
        for deck in data:
            print(deck, " ", data[deck])
        return None
    
    
    def prob_report_full_leaves(self, depth:int):
        """
        for each element s of S_n, find the probability that a full node at _depth_ is s
        """
        self.expand(depth)
        data = {str(list(deck)):0 for deck in it.permutations(range(1, self.n + 1))}
        for node in self.get_leaves(depth, full=True):
            deck = str(node.deck)
            prod = 1
            while node is not None:
                prod *= node.probability
                node = node.parent
            data[deck] += prod
        total = sum(data.values())
        print("probability reaching full:", total)
        print()
        for deck in data:
            print(deck, " ", "{:<23}".format(str(data[deck])), data[deck] / total)
        return None
            
    
    def P(self, t:int, sigma:list):
        if t < self.n:
            raise BaseException(f"t must be at least n={self.n}")
        if sorted(sigma) != get_deck(self.n):
            raise BaseException("sigma must be a permutation of n cards")
        
        self.expand(t)
        
        count = 0
        total = 0
        for node in self.get_leaves(t, full=True):
            total += 1
            if node.deck == sigma:
                count += 1
        return count / total

___

In [8]:
tree = Tree(3, "modified_rtr_2")
tree.count_full_leaves(3)
print()
tree.prob_report_full_leaves(3)

[1, 2, 3]   4
[1, 3, 2]   2
[2, 1, 3]   2
[2, 3, 1]   0
[3, 1, 2]   0
[3, 2, 1]   0

probability reaching full: 0.06400000000000002

[1, 2, 3]   0.03200000000000001     0.5
[1, 3, 2]   0.016000000000000004    0.25
[2, 1, 3]   0.016000000000000004    0.25
[2, 3, 1]   0                       0.0
[3, 1, 2]   0                       0.0
[3, 2, 1]   0                       0.0


In [9]:
tree.print(max_depth=3)

_______
| | | |   depth: 0
|1|2|3|    prob: 1
‾‾‾‾‾‾‾
      _______
      | | |*|   depth: 1
      |2|3|1|    prob: 0.2
      ‾‾‾‾‾‾‾
            _______
            | |*|*|   depth: 2
            |3|1|2|    prob: 0.2
            ‾‾‾‾‾‾‾
                  _______
                  |*|*|*|   depth: 3
                  |1|2|3|    prob: 0.2
                  ‾‾‾‾‾‾‾
                  _______
                  |*| |*|   depth: 3
                  |2|3|1|    prob: 0.2
                  ‾‾‾‾‾‾‾
                  _______
                  | |*|*|   depth: 3
                  |3|2|1|    prob: 0.2
                  ‾‾‾‾‾‾‾
                  _______
                  |*|*|*|   depth: 3
                  |1|3|2|    prob: 0.2
                  ‾‾‾‾‾‾‾
                  _______
                  | |*|*|   depth: 3
                  |3|1|2|    prob: 0.2
                  ‾‾‾‾‾‾‾
            _______
            |*| | |   depth: 2
            |1|2|3|    prob: 0.2
            ‾‾‾‾‾‾‾
                  